In [ ]:
# DETECTION MODEL

In [ ]:
import torch
import os
import torchvision
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torchvision.transforms as T

In [ ]:
# CHECKING DATASET STRUCTURE

train_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/train"
eval_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/eval"
test_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/test"

train_folder=os.listdir(train_path)
eval_folder=os.listdir(eval_path)
test_folder=os.listdir(test_path)

print(f"Nmumber of training samples: {len(train_folder)}")
print(f"Nmumber of evaluation samples: {len(eval_folder)}")
print(f"Nmumber of testing samples: {len(test_folder)}")
print(f"Total number of samples: {len(train_folder)+len(test_folder)+len(eval_folder)}")

# CHECKING ERRORS FOR FUNCTION: get_bounding_box

for file in train_folder: # executed also for eval and test set with no Error raised
    numbers=file.split("-")
    values=numbers[3]

    # if len(values) != 31:                                                                   questo controllo non serve in quanto dipende da quante cifre ogni numero ha
    #     raise ValueError(f" 1 st exception: Invalid plate text format in filename: {file}")
    
    values_v2=values.split("&")

    if len(values_v2) != 5:
        raise ValueError(f"2 nd exception: Invalid plate text format in filename: {file}")
    
    values_v3=[]
    for i in range(len(values_v2)):
        if "_" in values_v2[i]:
            values_v3.append(values_v2[i].split("_"))

    if len(values_v3) != 3:
        raise ValueError(f"3 rd exception: Invalid plate text format in filename: {file}")
    
    t=[values_v2[0],values_v3[0],values_v3[1],values_v3[2],values_v2[-1]]
    final_values = [int(x) for item in t for x in (item if isinstance(item, list) else [item])]
    x_coords=[final_values[0],final_values[2],final_values[4],final_values[6]]
    y_coords=[final_values[1],final_values[3],final_values[5],final_values[7]]

    if len(x_coords) != 4:
        raise ValueError(f"4 th exception: Invalid plate text format in filename: {file}")

 # CHECKING ERRORS FOR FUNCTION: get_text
    
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

for file in train_folder: # executed also for eval and test set with no Error raised
        values=file.split("-")

        if len(values)!=7:
            raise ValueError(f"1 st exception occurred: Invalid plate text format in filename: {file}")
        
        text=str(values[4])

        # if len(text)!=7:                                                                                questo controllo non serve in quanto dipende da quante cifre ogni numero ha cifre diverse
        #     raise ValueError(f"2 nd exception occurred: Invalid plate text format in filename: {file}") 
        
        indices=text.split("_")

        if len(indices) != 8:
            raise ValueError(f"3 rd exception occurred: Invalid plate text format in filename: {file}")
        
        province_character=provinces[int(indices[0])]
        alphabet_character=alphabet[int(indices[1])]
        ads_charachters=[ads[int(i)] for i in indices[2:]]
        plate_text=province_character+alphabet_character+"".join(ads_charachters)

        if len(plate_text) != 8:
            raise ValueError(f"4 th exception occurred: Invalid plate text format in filename: {file}")

# ESEMPIO DI PROVA CON DUE FILE PER FUNZIONE: get_bounding_box

file="308069444444444444-91_94-8&424_589&564-578&564_27&553_8&431_589&424-0_0_3_24_32_25_32_25-100-241.jpg" # FILE CHE NON DA PROBLEMI CON LA LUNGHEZZA DI VALUES
numbers=file.split("-")
values=numbers[3]
values_v2=values.split("&")
values_v3=[]
for i in range(len(values_v2)):
    if "_" in values_v2[i]:
        values_v3.append(values_v2[i].split("_"))
t=[values_v2[0],values_v3[0],values_v3[1],values_v3[2],values_v2[-1]]
final_values = [int(x) for item in t for x in (item if isinstance(item, list) else [item])]
x_coords=[final_values[0],final_values[2],final_values[4],final_values[6]]
y_coords=[final_values[1],final_values[3],final_values[5],final_values[7]]
x_min = min(x_coords)
y_min = min(y_coords)
x_max = max(x_coords)
y_max = max(y_coords)

file2="0115711805556-88_269-312&388_484&456-484&445_314&456_312&396_481&388-0_0_3_24_28_28_28_28-154-41.jpg" # FILE CHE DA PROBLEMI CON LA LUNGHEZZA DI VALUES
numbers=file.split("-")
values=numbers[3]
values_v2=values.split("&")
values_v3=[]
for i in range(len(values_v2)):
    if "_" in values_v2[i]:
        values_v3.append(values_v2[i].split("_"))
t=[values_v2[0],values_v3[0],values_v3[1],values_v3[2],values_v2[-1]]
final_values = [int(x) for item in t for x in (item if isinstance(item, list) else [item])]
x_coords=[final_values[0],final_values[2],final_values[4],final_values[6]]
y_coords=[final_values[1],final_values[3],final_values[5],final_values[7]]
x_min = min(x_coords)
y_min = min(y_coords)
x_max = max(x_coords)
y_max = max(y_coords)

# ESEMPIO DI PROVA CON DUE FILE PER FUNZIONE: get_text

file="30237890625-87_88-197&472_449&567-449&553_201&567_197&472_448&477-10_2_3_24_31_33_26_24-192-75.jpg" # FILE CHE NON DA PROBLEMI CON LA LUNGHEZZA DI TEXT
values=file.split("-")
text=str(values[4])
indices=text.split("_")
if len(indices) != 8:
    raise ValueError(f"Invalid plate text format in filename: {file}")
province_character=provinces[int(indices[0])]
alphabet_character=alphabet[int(indices[1])]
ads_charachters=[ads[int(i)] for i in indices[2:]]
plate_text=province_character+alphabet_character+"".join(ads_charachters)
print(values,text,len(text),indices,plate_text)

file2="0115711805556-88_269-312&388_484&456-484&445_314&456_312&396_481&388-0_0_3_24_28_28_28_28-154-41.jpg" # FILE CHE DA PROBLEMI CON LA LUNGHEZZA DI TEXT
values=file2.split("-")
text=str(values[4])
indices=text.split("_")
if len(indices) != 8:
    raise ValueError(f"Invalid plate text format in filename: {file}")
province_character=provinces[int(indices[0])]
alphabet_character=alphabet[int(indices[1])]
ads_charachters=[ads[int(i)] for i in indices[2:]]
plate_text=province_character+alphabet_character+"".join(ads_charachters)
print(values,text,len(text),indices,plate_text)

In [ ]:
# NUOVO DATASET CON PIU ELEMENTI DI TRAIN E MENO DI TEST

path_train_og="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/train"
path_test_og="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/test"
path_train_tf="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data_transf/train"
path_test_tf="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data_transf/test"
folder_train_og=os.listdir(path_train_og)
folder_test_og=os.listdir(path_test_og)
folder_train_tf=os.listdir(path_train_tf)
folder_test_tf=os.listdir(path_test_tf)
print(f"number of original training samples: {len(folder_train_og)}")
print(f"number of original test samples: {len(folder_test_og)}")
print(f"number of new training samples: {len(folder_train_tf)}")
print(f"number of new training samples: {len(folder_test_tf)}")

In [ ]:
# BOUNDING BOX FUNCTION 

def get_bounding_box(file):
    numbers=file.split("-")
    values=numbers[3]
    values_v2=values.split("&")
    values_v3=[]
    for i in range(len(values_v2)):
        if "_" in values_v2[i]:
            values_v3.append(values_v2[i].split("_"))
    t=[values_v2[0],values_v3[0],values_v3[1],values_v3[2],values_v2[-1]]
    final_values = [int(x) for item in t for x in (item if isinstance(item, list) else [item])]
    x_coords=[final_values[0],final_values[2],final_values[4],final_values[6]]
    y_coords=[final_values[1],final_values[3],final_values[5],final_values[7]]
    x_min = min(x_coords)
    y_min = min(y_coords)
    x_max = max(x_coords)
    y_max = max(y_coords)
    
    return [float(x_min), float(y_min), float(x_max), float(y_max)]

In [ ]:
# INTERSECTION OVER UNION FUNCTION

def compute_IoU(box1, box2):

    box1=box1.squeeze()
    box2=box2.squeeze()

    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    area_of_intersection = max(0, xB - xA) * max(0, yB - yA)

    area_box1 = (box1[2] - box1[0]) * (box1[3]- box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

    IoU = area_of_intersection / float(area_box1 + area_box2 - area_of_intersection)

    return IoU

In [ ]:
# DATASET   

import os
import torch
from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

smaller_train_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/train-20250508T142228Z-1-001/train"
train_path2="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/train2"
eval_path2="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/eval2"
test_path2="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2"

class CCPD_Dataset(torch.utils.data.Dataset):
    
    def __init__(self, path, transforms=None):
        self.path = path
        self.transforms = transforms
        self.folder = os.listdir(path)
        self.images = [f for f in self.folder if f.endswith(".jpg")]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        file = self.images[idx]
        full_path = os.path.join(self.path, file)
        image = Image.open(full_path).convert("RGB")

        bbox = get_bounding_box(file)
        tensor_bbox = torch.tensor([bbox], dtype=torch.float32)
        label = torch.tensor([1], dtype=torch.int64)  

        target = {"boxes": tensor_bbox, "labels": label}

        if target["boxes"].shape[0] != target["labels"].shape[0]:
            raise ValueError(f"Mismatch in number of boxes and labels for file: {file}")

        if self.transforms:
            image = self.transforms(image)

        return image, target
    
def collate_fn(batch):
    return tuple(zip(*batch))

transform = transforms.ToTensor()
train_dataset = CCPD_Dataset(train_path2, transforms=transform)
eval_dataset = CCPD_Dataset(eval_path2, transforms=transform)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn, num_workers=0) # watch out for the batch size, it is set to 1 for compatibility with the model
eval_dataloader = DataLoader(eval_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn, num_workers=0)  # watch out for the batch size, it is set to 1 for compatibility with the model

# EXAMPLE OF THE OUTPUT:

image_path_example="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/train/04-90_267-158&448_542&553-541&553_162&551_158&448_542&450-0_1_3_24_27_33_30_24-99-116.jpg"
images, targets =next(iter(train_dataset))
print(f"images: {images}, targets: {targets}")



In [ ]:
# MODEL

import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torch.optim as optim

model = fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes=2)
model.rpn.post_nms_top_n_train = 500
model.rpn.post_nms_top_n_test = 500
model.roi_heads.detections_per_img = 50  
# device = torch.device("cuda" if torch.cuda.is_available else "cpu")
device="cpu"
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
best_boxes=[]
best_avg_iou = 0.0
num_epochs=1

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    count=1
    for images, targets in train_dataloader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        train_loss += losses.item()
        print(f"Batch: {count} Training Loss: {losses.item():.4f}")
        count+=1

    model.eval()
    total_iou = 0
    count = 0
    with torch.no_grad():
        for images, targets in eval_dataloader:
            images = [img.to(device) for img in images]
            outputs = model(images)

            for pred, target in zip(outputs, targets):
                pred_boxes = pred['boxes'].to(device)
                gt_boxes = target['boxes'].to(device)

                if len(pred_boxes) > 0:
                    best_iou = 0.0
                    best_box = None
                    for box, score in zip(pred['boxes'], pred['scores']):
                        if score > 0.0:
                            iou = compute_IoU(box, gt_boxes[0])
                            if iou > best_iou:
                                best_iou = iou
                                best_box = box
                    if best_iou > 0:
                        total_iou += best_iou
                        count += 1
                        best_boxes.append(best_box.cpu())

    avg_iou = total_iou / count if count > 0 else 0

    if avg_iou > best_avg_iou:
        best_avg_iou = avg_iou
        torch.save(model.state_dict(), "best_frcnn_model.pth")
        print(f"Saved best model with IoU: {best_avg_iou:.4f}")

    print(f"Average IoU on eval set: {avg_iou:.4f}")

In [ ]:
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch

# Load model and image
model = fasterrcnn_resnet50_fpn(num_classes=2)  
model.load_state_dict(torch.load('best_frcnn_model.pth'))
device="cpu"
model.to(device)
model.eval()
folder=os.listdir("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2")
for image in folder:
    if image.endswith(".jpg"):
        image_path = os.path.join("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2", image)
        image = Image.open(image_path).convert("RGB")
    transform = T.ToTensor()
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image_tensor)
        pred = output[0]
        score = output[0]['scores'][0].item()
        print(score)

In [ ]:
# TESTING THE TRAINED MODEL USING THE BEST BOUNDING BOX

from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch

# Load model and image
model = fasterrcnn_resnet50_fpn(num_classes=2)  
model.load_state_dict(torch.load('best_frcnn_model.pth'))
device="cpu"
model.to(device)
model.eval()
folder=os.listdir("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2")
for image in folder:
    if image.endswith(".jpg"):
        image_path = os.path.join("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2", image)
        image = Image.open(image_path).convert("RGB")
    transform = T.ToTensor()
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image_tensor)
        pred = output[0]
        score = output[0]['scores'][0].item()        
        best_bb=pred['boxes'][0].cpu()
        gt_box=get_bounding_box(image_path)
        tensor_bbox = torch.tensor([gt_box], dtype=torch.float32)
        iou=compute_IoU(best_bb, tensor_bbox)
        plt.imshow(image)
        ax = plt.gca()
        x1, y1, x2, y2 = best_bb
        rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)
        ax.text(x1, y1, f" score: {score:.2f}, IoU: {iou:.2f}", color='white', bbox=dict(facecolor='red', alpha=0.5))
        plt.axis('off')
        plt.show()

In [ ]:
# TESTING THE UNTRAINED MODEL USING THE BEST BOUNDING BOX

from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch

# Load model and image
model = fasterrcnn_resnet50_fpn(num_classes=2)  
device="cpu"
model.to(device)
model.eval()
folder=os.listdir("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2")
for image in folder:
    if image.endswith(".jpg"):
        image_path = os.path.join("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2", image)
        image = Image.open(image_path).convert("RGB")
    transform = T.ToTensor()
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image_tensor)
        pred = output[0]
        score = output[0]['scores'][0].item()        
        best_bb=pred['boxes'][0].cpu()
        gt_box=get_bounding_box(image_path)
        tensor_bbox = torch.tensor([gt_box], dtype=torch.float32)
        iou=compute_IoU(best_bb, tensor_bbox)
        plt.imshow(image)
        ax = plt.gca()
        x1, y1, x2, y2 = best_bb
        rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)
        ax.text(x1, y1, f" score: {score:.2f}, IoU: {iou:.2f}", color='white', bbox=dict(facecolor='red', alpha=0.5))
        plt.axis('off')
        plt.show()

In [ ]:
# COMPUTING AVERAGE IOU ON TEST SET USING THE TRAINED MODEL

from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch

# Load model and image
model = fasterrcnn_resnet50_fpn(num_classes=2)  
model.load_state_dict(torch.load('best_frcnn_model.pth'))
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
folder=os.listdir("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data_transf/test")
total_iou=0
for image in folder:
    if image.endswith(".jpg"):
        image_path = os.path.join("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data_transf/test", image)
        image = Image.open(image_path).convert("RGB")
    transform = T.ToTensor()
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image_tensor)
        pred = output[0]
        if len(output[0]['scores'])>0:
            score = output[0]['scores'][0].item()        
            best_bb=pred['boxes'][0].to(device)
            gt_box=get_bounding_box(image_path)
            tensor_bbox = torch.tensor([gt_box], dtype=torch.float32)
            iou=compute_IoU(best_bb, tensor_bbox)
            total_iou+=iou
print(f"Average IoU: {total_iou/len(folder)}") # 0.58

In [ ]:
# CROP FUNCTION WITH PREDICTED BOUNDING BOX

def crop_image_with_RCNN(file):
    
    image = Image.open(file).convert("RGB")
    transform = T.ToTensor()
    img_tensor = transform(image).unsqueeze(0).to(device) 
    with torch.no_grad():
        prediction = model(img_tensor)[0]
        best_bb=prediction['boxes'][0].to(device)
        best_bb=best_bb.int()
        cropped_image = img_tensor[0, :, best_bb[1]:best_bb[3], best_bb[0]:best_bb[2]]
    return cropped_image

def crop_folder_with_RCNN(folder_path):
    cropped_folder = []
    files = os.listdir(folder_path)
    for file in files:
        full_path = os.path.join(folder_path, file)
        cropped_image = crop_image_with_RCNN(full_path)
        cropped_folder.append(cropped_image)
    return cropped_folder

def crop_image_with_ground_truth(full_path):
    filename = os.path.basename(full_path)  
    bb = get_bounding_box(filename)
    image = Image.open(full_path).convert("RGB")
    cropped_image = image.crop(bb)
    return cropped_image

def crop_folder_with_ground_truth(folder_path):
    cropped_folder = []
    files = os.listdir(folder_path)
    for file in files:
        full_path = os.path.join(folder_path, file)
        cropped_image = crop_image_with_ground_truth(full_path)
        cropped_folder.append(cropped_image)
    return cropped_folder

In [ ]:
# SHOWING THE RESULTS OF THE TRAINED MODEL

for image in os.listdir("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data_transf/test"):
    if image.endswith(".jpg"):
        image_path = os.path.join("/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data_transf/test", image)
        cropped = crop_image_with_RCNN(image_path)
        plt.imshow(cropped.permute(1, 2, 0).cpu().numpy())
        plt.axis("off")
        plt.title("Cropped Image")
        plt.show()

In [ ]:
# ------------------------
# SETUP & IMPORTS
# ------------------------
import os
import gc
import torch
import torchvision
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.cuda.amp import GradScaler, autocast
import torch.optim as optim

# ------------------------
# DATASET
# ------------------------


class ResizeTransform:
    def __call__(self, image):
        return transforms.functional.resize(image, [512, 512])

class CCPD_Dataset(torch.utils.data.Dataset):
    def __init__(self, path, transforms=None):
        self.path = path
        self.transforms = transforms
        self.images = [f for f in os.listdir(path) if f.endswith(".jpg")]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        file = self.images[idx]
        full_path = os.path.join(self.path, file)
        image = Image.open(full_path).convert("RGB")
        bbox = get_bounding_box(file)
        tensor_bbox = torch.tensor([bbox], dtype=torch.float32)
        label = torch.tensor([1], dtype=torch.int64)
        target = {"boxes": tensor_bbox, "labels": label}

        if self.transforms:
            image = self.transforms(image)

        return image, target

def collate_fn(batch):
    return tuple(zip(*batch))

# ------------------------
# DATA LOADING
# ------------------------

train_path='/content/drive/My Drive/CV Project/CCPD2020/ccpd_green/train'
eval_path='/content/drive/My Drive/CV Project/CCPD2020/ccpd_green/val'

transform = transforms.Compose([
    ResizeTransform(),
    transforms.ToTensor()
])

train_dataset = CCPD_Dataset(train_path, transforms=transform)
eval_dataset = CCPD_Dataset(eval_path, transforms=transform)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn, num_workers=0)
eval_dataloader = DataLoader(eval_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=0)

# train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
# eval_dataloader = DataLoader(eval_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

# ------------------------
# MODEL
# ------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = fasterrcnn_resnet50_fpn(pretrained=True)

# Custom head
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes=2)

# Reduce number of proposals
model.rpn.post_nms_top_n_train = 100
model.rpn.post_nms_top_n_test = 100
model.roi_heads.detections_per_img = 50

model.to(device)

# Optimizer & mixed precision
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
scaler = GradScaler()

# ------------------------
# TRAINING LOOP
# ------------------------


best_avg_iou = 0.0
best_boxes = []

for epoch in range(1):
    print(f"\nEpoch {epoch+1}")
    model.train()
    train_loss = 0
    for i, (images, targets) in enumerate(train_dataloader):
        gc.collect()
        torch.cuda.empty_cache()

        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        with autocast():
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        scaler.scale(losses).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += losses.item()
        print(f"Batch {i+1} - Loss: {losses.item():.4f}")

    print(f"Epoch {epoch+1} Total Training Loss: {train_loss:.4f}")

    # ------------------------
    # EVALUATION
    # ------------------------
    model.eval()
    total_iou = 0
    count = 0
    with torch.no_grad():
        for images, targets in eval_dataloader:
            images = [img.to(device) for img in images]
            outputs = model(images)

            for pred, target in zip(outputs, targets):
                pred_boxes = pred['boxes'].to(device)
                gt_boxes = target['boxes'].to(device)

                if len(pred_boxes) > 0:
                    best_iou = 0.0
                    best_box = None
                    for box, score in zip(pred['boxes'], pred['scores']):
                        if score > 0.0:
                            iou = compute_IoU(box, gt_boxes[0])
                            if iou > best_iou:
                                best_iou = iou
                                best_box = box
                    if best_iou > 0:
                        total_iou += best_iou
                        count += 1
                        best_boxes.append(best_box.cpu())

    avg_iou = total_iou / count if count > 0 else 0
    print(f"Average IoU: {avg_iou:.4f}")

    # Save best model
    if avg_iou > best_avg_iou:
        best_avg_iou = avg_iou
        torch.save(model.state_dict(), "/content/drive/My Drive/best_frcnn_model.pth")
        print(f"Saved best model with IoU: {best_avg_iou:.4f}")


In [ ]:
def visualize_detection(image_path, boxes, scores):
    """
    Visualize detected license plates on the image
    
    Args:
        image_path: Path to the original image
        boxes: Detected bounding boxes
        scores: Confidence scores for each detection
    """
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    
    # Load image
    image = Image.open(image_path)
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.imshow(image)
    
    # Add bounding boxes
    for box, score in zip(boxes, scores):
        x1, y1, x2, y2 = box
        rect = patches.Rectangle(
            (x1, y1), x2-x1, y2-y1,
            linewidth=2, edgecolor='r', facecolor='none'
        )
        ax.add_patch(rect)
        ax.text(x1, y1-10, f'Conf: {score:.2f}', color='red')
    
    plt.axis('off')
    plt.show()

In [ ]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from utils import detect_license_plate, visualize_detection

# 1. Load the trained model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = fasterrcnn_resnet50_fpn(num_classes=2)  # 2 classes: background and license plate
model.load_state_dict(torch.load('best_frcnn_model.pth'))
model.to(device)

# 2. Run inference on an image
image_path = 'path/to/your/image.jpg'
boxes, scores = detect_license_plate(model, image_path, device=device, confidence_threshold=0.5)

# 3. Visualize the results
visualize_detection(image_path, boxes, scores)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as T
import torch

model.eval()

img_path = "/content/drive/My Drive/CV Project/CCPD2020/ccpd_green/train_small/Copia di 015-90_260-228&437_444&507-444&507_238&502_228&439_435&437-0_0_3_24_28_26_28_32-111-64.jpg"
image = Image.open(img_path).convert("RGB")
transform = T.ToTensor()
img_tensor = transform(image).unsqueeze(0).to(device)
with torch.no_grad():
    prediction = model(img_tensor)
plt.figure(figsize=(10, 8))
plt.imshow(image)  
ax = plt.gca()
if len(best_boxes) > 1:
    x1, y1, x2, y2 = best_boxes[1]
    rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1,
                         fill=False, edgecolor='red', linewidth=2)
    ax.add_patch(rect)
plt.title("Predicted Car Plate Bounding Box")
plt.axis("off")
plt.show()

In [ ]:
# RECOGNITION MODEL

In [ ]:
import torch
import os
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms as T
from torch.utils.data import Dataset

In [ ]:
# CAR PLATE TEXT FUNCTION

provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

def get_text(file):
    values=file.split("-")
    text=str(values[4])
    indices=text.split("_")
    province_character=provinces[int(indices[0])]
    alphabet_character=alphabet[int(indices[1])]
    ads_charachters=[ads[int(i)] for i in indices[2:]]
    plate_text=province_character+alphabet_character+"".join(ads_charachters)
    return plate_text

In [ ]:
FUNCTION = "with_ground_truth"  

match FUNCTION:
    case "with_ground_truth":
        selected_function = crop_folder_with_ground_truth
    case "with_RCNN":
        selected_function = crop_folder_with_RCNN

In [ ]:
# DATASET

all_characters = sorted(set(provinces + alphabet + ads))
char2idx = {char: idx + 1 for idx, char in enumerate(all_characters)}  

class CCPD_dataset_recognition(Dataset):
    
    def __init__(self, path, char2idx, transformations):

        self.path = path
        self.char2idx = char2idx
        self.transformations = transformations
        self.images = [f for f in os.listdir(path) if f.endswith("jpg")]
        self.cropped_images = selected_function(self.path)  

    def __len__(self):

        return len(self.images)
    
    def __getitem__(self, idx):

        file = self.images[idx]
        image = self.cropped_images[idx]
        gt_text = get_text(file)
        gt_text_encoded = torch.tensor([self.char2idx[c] for c in gt_text], dtype=torch.long)
        if self.transformations:
            image = self.transformations(image)
        return image, gt_text_encoded
    
def crnn_collate_fn(batch):
    images, labels = zip(*batch)
    image_batch = torch.stack(images)
    label_lengths = torch.tensor([len(label) for label in labels], dtype=torch.long)
    labels_concat = torch.cat(labels)
    return image_batch, labels_concat, label_lengths

transformations=T.Compose([T.Grayscale(num_output_channels=1),T.Resize((64,256)),T.ToTensor()])

train_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/train"
eval_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/eval"

train_recognition_dataset=CCPD_dataset_recognition(train_path,char2idx,transformations=transformations)
eval_recognition_dataset=CCPD_dataset_recognition(eval_path,char2idx,transformations=transformations)

train_dataloader=DataLoader(train_recognition_dataset,batch_size=16, shuffle=True, collate_fn=crnn_collate_fn)
eval_dataloader=DataLoader(eval_recognition_dataset,batch_size=8, shuffle=False, collate_fn=crnn_collate_fn)

batch=next(iter(train_dataloader))
batch
# returns a batch containing the images as tensors and the ground truth of the text once treated with char2idx (all stacked)

In [ ]:
# MODEL

import torch.nn as nn

class CRNN(nn.Module):
    def __init__(self, num_classes):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.AdaptiveAvgPool2d((2, None)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
        )

        self.rnn = nn.LSTM(1024, 256, bidirectional=True, num_layers=2,dropout=0.5, batch_first=True)
        self.fc = nn.Linear(512, num_classes + 1)  # +1 for CTC blank

    def forward(self, x):
        
        x = self.cnn(x)                          # [B, 512, 2, W]
        b, c, h, w = x.size()
        assert h == 2, f"Expected height to be 2 after CNN, got {h}"
        x = x.permute(0, 3, 1, 2).contiguous()   # [B, W, C, H]
        x = x.view(b, w, c * h)                  # [B, W, 1024]
        x, _ = self.rnn(x)                       # [B, W, 512]
        x = self.fc(x)                           # [B, W, num_classes + 1]
        return x.permute(1, 0, 2)                # [T, B, C] for CTC
    
recognition_model=CRNN(67)
recognition_model(batch[0]).shape
# returns a tensor of size 64X16X68 --> this output has to be decoded

In [ ]:
# TRAINING AND EVALUATION LOOP

from torch.nn import CTCLoss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = recognition_model.to(device)

loss_fn = CTCLoss(blank=0, zero_infinity=True)
optimizer = torch.optim.Adam(recognition_model.parameters(), lr=0.001)
epochs = 10
best_loss = float('inf')

for epoch in range(epochs):
    print(f'Epoch: {epoch + 1}\n-------')
    recognition_model.train()
    train_loss = 0.0

    for batch, (images, targets, target_lengths) in enumerate(train_dataloader):
        images = images.to(device)
        targets = targets.to(device)
        target_lengths = target_lengths.to(device)
        preds = recognition_model(images)  # preds: [T, B, C]
        T, B, C = preds.size()
        input_lengths = torch.full(size=(B,), fill_value=T, dtype=torch.long)  # Each sample has T time steps
        log_probs = preds.log_softmax(2)  # dim=2 is for classes

        # Compute loss
        loss = loss_fn(log_probs, targets, input_lengths, target_lengths)
        train_loss += loss.item()

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            print(f'Batch {batch}: Loss = {loss.item():.4f}')

    avg_loss = train_loss / len(train_dataloader)
    print(f'Average training loss: {avg_loss:.4f}')

model.eval()
test_loss = 0.0

with torch.inference_mode():
    for images, targets, target_lengths in eval_dataloader:
        preds = model(images)
        log_probs = preds.log_softmax(2)
        T, B, C = preds.size()
        input_lengths = torch.full(size=(B,), fill_value=T, dtype=torch.long)
        loss = loss_fn(log_probs, targets, input_lengths, target_lengths)
        test_loss += loss.item()

    test_loss /= len(eval_dataloader)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), "best_crnn_ctc_model.pth")

    print(f'Evaluation loss: {test_loss:.4f}')

In [ ]:
# DECODING RESULTS

idx2char = {v: k for k, v in char2idx.items()}

def greedy_decode(ctc_output, idx2char):
    # ctc_output: [T, B, C] (output from the model)
    preds = ctc_output.permute(1, 0, 2)  # [B, T, C]
    pred_strings = []
    for pred in preds:
        best_path = torch.argmax(pred, dim=1).tolist()  # get index with max prob at each timestep
        prev = -1
        decoded = []
        for p in best_path:
            if p != prev and p != 0:  # 0 is the CTC blank
                decoded.append(idx2char[p])
            prev = p
        pred_strings.append("".join(decoded))
    return pred_strings

recognition_model.eval()
correct = 0
total = 0

with torch.inference_mode():
    for images, targets, target_lengths in eval_dataloader:
        outputs = recognition_model(images)  # [T, B, C]
        predictions = greedy_decode(outputs, idx2char)

        # Compare to original labels (you need original ground truth strings)
        # This depends on how your DataLoader is structured

        # Example (if you stored gt_texts during dataset loading):
        for pred, true in zip(predictions, gt_texts):
            if pred == true:
                correct += 1
            total += 1

print(f"Accuracy: {correct / total * 100:.2f}%")
